Convert parquet file to sqlute databse

In [1]:
import pandas as pd
import sqlite3

# Read the Parquet file into a DataFrame
df = pd.read_parquet('prompts.parquet')

# Ensure there is a primary key column; if not, add one
if 'id' not in df.columns:
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'id'}, inplace=True)

# Connect to a SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect('prompts.db')
cursor = conn.cursor()

# Create a metadata table
cursor.execute('''
CREATE TABLE IF NOT EXISTS metadata (
    key TEXT PRIMARY KEY,
    value TEXT
)
''')

# Insert some metadata
cursor.execute('''
INSERT OR REPLACE INTO metadata (key, value) VALUES
('version', '1.0'),
('description', 'This is the train table from the prompts.parquet file')
''')

# Write the DataFrame to a SQLite table
df.to_sql('prompts_table', conn, if_exists='replace', index=False)

# Add a primary key constraint to the train_table
cursor.execute('''
CREATE TABLE IF NOT EXISTS prompts_table_temp AS SELECT * FROM prompts_table;
DROP TABLE prompts_table;
CREATE TABLE prompts_table (
    id INTEGER PRIMARY KEY,
    /* include other columns here explicitly if needed, e.g., col1, col2, ... */
);
INSERT INTO prompts_table SELECT * FROM prompts_table_temp;
DROP TABLE prompts_table_temp;
''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("Parquet file has been successfully converted to a SQLite database with additional metadata.")


Warning: You can only execute one statement at a time.